# 1. Import libraries

In [8]:
import tensorflow as tf
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt

keras = tf.keras
t = Okt()

# 2. fit tokenizer to our datasets

In [9]:
from vectorizer import BaseVectorizer
tokenizer = BaseVectorizer(t.morphs)

In [12]:
df = pd.read_csv('train_intent.csv',encoding='CP949')

In [14]:
df[0:1]

,question,intent
0,인터넷 켜줘,internet


In [15]:
tokenizer.fit(df['question'].values)

scanning was done                                        
64 terms are recognized


In [16]:
tokenizer.vocabulary_

{'_PAD_': 0,
 '_UNK_': 1,
 '_STA_': 2,
 '_EOS_': 3,
 '인터넷': 4,
 '그림': 5,
 '봐': 6,
 '켜': 7,
 '좀': 8,
 '그리고': 9,
 '나': 10,
 '뭐': 11,
 '그릴': 12,
 '그림판': 13,
 '할래': 14,
 '크롬': 15,
 '틀어': 16,
 '그': 17,
 '리자': 18,
 '싶어': 19,
 '켜줘': 20,
 '이나': 21,
 '한': 22,
 '래': 23,
 '컴퓨터': 24,
 '로': 25,
 '알아보게': 26,
 '구글': 27,
 '음': 28,
 '이': 29,
 '싶다': 30,
 '만': 31,
 '거': 32,
 '오늘': 33,
 '은': 34,
 '그려': 35,
 '보자': 36,
 '지금': 37,
 '심심한데': 38,
 '홈페이지': 39,
 '띄워': 40,
 '하자': 41,
 '번': 42,
 '할거야': 43,
 '그릴거야': 44,
 '틀어줘': 45,
 '그려야': 46,
 '징': 47,
 '그려야지': 48,
 '하고싶은데': 49,
 '하고': 50,
 '할': 51,
 '수': 52,
 '있나': 53,
 '하고싶네': 54,
 '할까': 55,
 '웹서핑': 56,
 '검색': 57,
 '하게': 58,
 '그리게': 59,
 '싶은데': 60,
 '싶네': 61,
 '아': 62,
 '그려야겠다': 63}

# 3. data preprocessing

In [17]:
label_to_id = {t:i for i,t in enumerate(df.intent.unique())}
id_to_label = {i:t for i,t in enumerate(df.intent.unique())}

In [18]:
print(label_to_id)

{'internet': 0, 'paint': 1}


In [19]:
# df.intent = df.intent.map(lambda x : label_index[x])
# print(df.head(10))

In [22]:
MAX_LENGTH = 40
def tokenize_and_filter(sentences, labels):
    inputs, outputs = [], []
  
    for sentence, label in zip(sentences, labels):
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
            outputs.append(label_to_id[label])
  
      # pad tokenized sentences
        padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
            inputs, maxlen=MAX_LENGTH, padding='post', 
            value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
    return padded_inputs, outputs

In [23]:
inputs, outputs = tokenize_and_filter(df.question, df.intent)

In [24]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

encoded input :  [ 4 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] label :  0 original input sentence :  ['인터넷', '켜줘', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_']


In [25]:
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [26]:
for x, y in dataset.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

tf.Tensor(
[[ 5 48  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [33 34  5 21 35 36  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5 46 47  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [28  4 21 55  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [15  7  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [62 63  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  9 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0

# 4. model design

In [27]:
print(len(label_to_id.values()))

2


In [45]:
def get_model():
    return tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])

In [46]:
model = get_model()

In [47]:
LEARNING_RATE = 0.0001

In [48]:
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [50]:
model.fit(dataset, epochs=20)

Epoch 1/20
4/4 [==============================] - 0s 17ms/step - loss: 0.6761 - sparse_categorical_accuracy: 0.6792
Epoch 2/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6753 - sparse_categorical_accuracy: 0.6604
Epoch 3/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6744 - sparse_categorical_accuracy: 0.6226
Epoch 4/20
4/4 [==============================] - 0s 16ms/step - loss: 0.6725 - sparse_categorical_accuracy: 0.6415
Epoch 5/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6716 - sparse_categorical_accuracy: 0.7358
Epoch 6/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6695 - sparse_categorical_accuracy: 0.8302
Epoch 7/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6680 - sparse_categorical_accuracy: 0.8302
Epoch 8/20
4/4 [==============================] - 0s 15ms/step - loss: 0.6627 - sparse_categorical_accuracy: 0.8302
Epoch 9/20
4/4 [==============================] - 0s 15ms/step - loss: 0

In [51]:
def question_processing(sentences):
    inputs = []
    for sentence in sentences:
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')
    # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [52]:
input_sentence = question_processing(['인터넷 한 번 켜봐', 
                                      '그림 그리게 인터넷 켜줘',
                                      '인터넷이나 한 번 해볼까',
                                      '음.. 그림 그려볼까',
                                      '인터넷 ㄱㄱ'
                                     ])

In [53]:
model.predict(input_sentence)

array([[0.53525996, 0.46474007],
       [0.48697442, 0.51302564],
       [0.52566403, 0.4743359 ],
       [0.47329047, 0.52670956],
       [0.51432174, 0.48567832]], dtype=float32)

In [54]:
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

[0 1 0 1 0]


In [55]:
for p in prediction:
    print(id_to_label[p])

internet
paint
internet
paint
internet


In [ ]:
# # 모델 저장하기
# model.save('entitytestmodel.h5')
# 모델 불러오기
tempmodel = keras.models.load_model('intent_model.h5')